In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

### 2.11

In [4]:
ratings = pd.read_csv('ml-1m/ratings.dat',
                      sep='\n',
                      delimiter='::',
                      header=None,
                      names=['UserID', 'MovieID','Rating','Timestamp'],
                      engine='python')


In [5]:
movies = pd.read_csv('ml-1m/movies.dat',sep='\n',
                     delimiter='::',
                     header=None, 
                     names=['MovieID', 'Title','Genres'], 
                     engine='python')

In [6]:
users = pd.read_csv('ml-1m/users.dat',sep='\n',
                     delimiter='::',
                     header=None, 
                     names=['UserID', 'Gender','Age', 'Occupation','Zip-code'], 
                     engine='python')

In [7]:
users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [8]:
movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [10]:
movies.set_index("MovieID")
movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [11]:
movies[["Title", "Year"]] = movies.Title.str.extract(r"(.*) \((\d{4})\)")

movies.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


,MovieID,Title,Genres,Year
0,1,Toy Story,Animation|Children's|Comedy,1995
1,2,Jumanji,Adventure|Children's|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama,1995
4,5,Father of the Bride Part II,Comedy,1995


In [12]:
movies['MovieID'].count()

3883

In [13]:
movies['Year']  = movies['Year'].astype(int)

In [14]:
countByYear = movies.groupby('Year').count()
countByYear.head()

,MovieID,Title,Genres
Year,,,
1919,3,3,3
1920,2,2,2
1921,1,1,1
1922,2,2,2
1923,3,3,3


In [15]:
countByYear = movies.Year.value_counts(sort=True)
countByYear.head()

1996    345
1995    342
1998    337
1997    315
1999    283
Name: Year, dtype: int64

In [16]:
sexRatio = pd.crosstab(users.Gender, users.Age)
sexRatio

Age,1,18,25,35,45,50,56
Gender,,,,,,,
F,78,298,558,338,189,146,102
M,144,805,1538,855,361,350,278


In [17]:
allGenresAccurances = pd.Series(np.concatenate(movies.Genres.str.split('|'), axis=0))
allGenresAccurances.columns = ['Genre']
allGenresAccurances.value_counts(sort=True)

Drama          1603
Comedy         1200
Action          503
Thriller        492
Romance         471
Horror          343
Adventure       283
Sci-Fi          276
Children's      251
Crime           211
War             143
Documentary     127
Musical         114
Mystery         106
Animation       105
Western          68
Fantasy          68
Film-Noir        44
dtype: int64

In [20]:
ratings_by_movie = pd.merge(movies,ratings, left_index = True, right_on="MovieID").groupby('Title').mean()['Rating'].sort_values(ascending=False)
ratings_by_movie = ratings_by_movie.reset_index()
ratings_by_movie.head()

,Title,Rating
0,"Perils of Pauline, The",5.0
1,Beautiful People,5.0
2,"Cup, The (Ph�rpa)",5.0
3,American Pop,5.0
4,Passion in the Desert,5.0


In [22]:
ratings_count = pd.merge(movies,ratings, left_index=True,right_on='MovieID').groupby('Title').count()['Rating'] 
movies_with_100_ratings = pd.Series(ratings_count[ratings_count > 100]).reset_index()

pd.merge(movies_with_100_ratings,ratings_by_movie, on='Title').sort_values('Rating_y', ascending=False).head()

,Title,Rating_x,Rating_y
1369,Popeye,628,4.560510
1692,Strawberry and Chocolate (Fresa y chocolate),2227,4.554558
941,Kansas City,2223,4.524966
959,Kim,657,4.520548
746,Guardian Angel,1783,4.517106


In [36]:
user_ratings = pd.merge(users.set_index('UserID'),ratings, left_index=True, right_on='UserID').set_index('MovieID')
user_ratings.head()

,Gender,Age,Occupation,Zip-code,UserID,Rating,Timestamp
MovieID,,,,,,,
1193,F,1,10,48067,1,5,978300760
661,F,1,10,48067,1,3,978302109
914,F,1,10,48067,1,3,978301968
3408,F,1,10,48067,1,4,978300275
2355,F,1,10,48067,1,5,978824291


In [62]:
user_ratings_movies = pd.merge(user_ratings, movies, left_index=True, right_on='MovieID')
best_movies_by_gender = user_ratings_movies.groupby(['Gender', 'Title']).agg(['mean', 'count'])['Rating']
best_movies_by_gender = best_movies_by_gender[best_movies_by_gender['count'] > 100]
best_movies_by_gender.reset_index().sort_values(by=['mean','Gender'], ascending=False).drop_duplicates(['Gender'])


,Gender,Title,mean,count
162,F,"Close Shave, A",4.644444,180
1450,M,"Godfather, The",4.583333,1740


In [76]:
mean_year_by_age = user_ratings_movies.groupby(['Age']).agg(['mean'])['Year']
np.round(mean_movies_by_age)

,mean
Age,
1,1989.0
18,1990.0
25,1988.0
35,1985.0
45,1984.0
50,1982.0
56,1982.0


In [89]:
user_ratings_movies_female = user_ratings_movies[user_ratings_movies['Gender'] == 'F']
user_ratings_movies_male = user_ratings_movies[user_ratings_movies['Gender'] == 'M']

print(pd.Series(user_ratings_movies_female.Genres.str.split(pat = '|', expand=True).values.ravel()).describe().iloc[2])
print(pd.Series(user_ratings_movies_male.Genres.str.split(pat = '|', expand=True).values.ravel()).describe().iloc[2])

Drama
Comedy


# 2.10

In [5]:
import os

In [6]:
import sqlite3

In [9]:
flights = pd.read_csv('nycflights13_flights.csv.gz',
                      comment ='#',
                      engine='python')


In [10]:
planes = pd.read_csv('nycflights13_planes.csv.gz',
                      comment ='#',
                      engine='python')


In [11]:
weather = pd.read_csv('nycflights13_weather.csv.gz',
                      comment ='#',
                      engine='python')

In [12]:
airports = pd.read_csv('nycflights13_airports.csv.gz',
                      comment ='#',
                      engine='python')

In [13]:
airlines = pd.read_csv('nycflights13_airlines.csv.gz',
                      comment ='#',
                      engine='python')

In [14]:
db_name = 'nycflights13.db'
os.remove(os.path.join(os.getcwd(), db_name))

In [15]:
db_name = 'nycflights13.db'
con = sqlite3.connect(os.path.join(os.getcwd(), db_name))
planes.to_sql('planes', con);
flights.to_sql('flights', con);
airlines.to_sql('airlines', con);
weather.to_sql('weather', con);
airports.to_sql('airports', con);

In [46]:
sql_result3 = pd.read_sql_query('Select count(*), engine from planes group by engine', con)
sql_result3

,count(*),engine
0,2,4 Cycle
1,28,Reciprocating
2,2750,Turbo-fan
3,535,Turbo-jet
4,2,Turbo-prop
5,5,Turbo-shaft


In [47]:
frame3 = pd.DataFrame(planes.groupby('engine').count()['manufacturer']).reset_index()[['manufacturer', 'engine']]
frame3.columns = ['count(*)', 'engine']
frame3


,count(*),engine
0,2,4 Cycle
1,28,Reciprocating
2,2750,Turbo-fan
3,535,Turbo-jet
4,2,Turbo-prop
5,5,Turbo-shaft


In [50]:
sql_result4 = pd.read_sql_query('select count(*),engine, type from planes group by engine,type', con)
sql_result4

,count(*),engine,type
0,2,4 Cycle,Fixed wing single engine
1,5,Reciprocating,Fixed wing multi engine
2,23,Reciprocating,Fixed wing single engine
3,2750,Turbo-fan,Fixed wing multi engine
4,535,Turbo-jet,Fixed wing multi engine
5,2,Turbo-prop,Fixed wing multi engine
6,5,Turbo-shaft,Rotorcraft


In [79]:
frame4 = pd.DataFrame(planes.groupby(['engine', 'type']).count()['manufacturer']).reset_index()
frame4.columns = ['engine','type','count(*)']
frame4 = frame4[['count(*)', 'engine', 'type']]
frame4

,count(*),engine,type
0,2,4 Cycle,Fixed wing single engine
1,5,Reciprocating,Fixed wing multi engine
2,23,Reciprocating,Fixed wing single engine
3,2750,Turbo-fan,Fixed wing multi engine
4,535,Turbo-jet,Fixed wing multi engine
5,2,Turbo-prop,Fixed wing multi engine
6,5,Turbo-shaft,Rotorcraft


In [91]:
frame4b = pd.crosstab(planes.engine, planes.type).unstack().reset_index()
frame4b = frame4b[frame4b[0] != 0]
frame4b.columns = ['type', 'engine', 'count(*)']
frame4b.reset_index()[['count(*)','engine', 'type']]

,count(*),engine,type
0,5,Reciprocating,Fixed wing multi engine
1,2750,Turbo-fan,Fixed wing multi engine
2,535,Turbo-jet,Fixed wing multi engine
3,2,Turbo-prop,Fixed wing multi engine
4,2,4 Cycle,Fixed wing single engine
5,23,Reciprocating,Fixed wing single engine
6,5,Turbo-shaft,Rotorcraft


In [77]:
sql_result5 = pd.read_sql_query('select min(year), avg(year), max(year),engine,manufacturer from planes group by engine, manufacturer',con)
sql_result5

,min(year),avg(year),max(year),engine,manufacturer
0,1975.0,1975.000000,1975.0,4 Cycle,CESSNA
1,NaN,NaN,NaN,4 Cycle,JOHN G HESS
2,NaN,NaN,NaN,Reciprocating,AMERICAN AIRCRAFT INC
3,2007.0,2007.000000,2007.0,Reciprocating,AVIAT AIRCRAFT INC
4,NaN,NaN,NaN,Reciprocating,BARKER JACK L
5,1959.0,1971.142857,1983.0,Reciprocating,CESSNA
6,2007.0,2007.000000,2007.0,Reciprocating,CIRRUS DESIGN CORP
7,1959.0,1959.000000,1959.0,Reciprocating,DEHAVILLAND
8,1956.0,1956.000000,1956.0,Reciprocating,DOUGLAS
9,2007.0,2007.000000,2007.0,Reciprocating,FRIEDEMANN JON


In [101]:
frame5 = pd.DataFrame(planes.groupby(['engine', 'manufacturer']).agg(['min','mean','max'])['year']).reset_index()
frame5.columns = ['engine','manufacturer','min(year)','avg(year)','max(year)']
frame5[['min(year)','avg(year)','max(year)','engine','manufacturer']]

,min(year),avg(year),max(year),engine,manufacturer
0,1975.0,1975.000000,1975.0,4 Cycle,CESSNA
1,NaN,NaN,NaN,4 Cycle,JOHN G HESS
2,NaN,NaN,NaN,Reciprocating,AMERICAN AIRCRAFT INC
3,2007.0,2007.000000,2007.0,Reciprocating,AVIAT AIRCRAFT INC
4,NaN,NaN,NaN,Reciprocating,BARKER JACK L
5,1959.0,1971.142857,1983.0,Reciprocating,CESSNA
6,2007.0,2007.000000,2007.0,Reciprocating,CIRRUS DESIGN CORP
7,1959.0,1959.000000,1959.0,Reciprocating,DEHAVILLAND
8,1956.0,1956.000000,1956.0,Reciprocating,DOUGLAS
9,2007.0,2007.000000,2007.0,Reciprocating,FRIEDEMANN JON


In [102]:
sql_result6 = pd.read_sql_query('select * from planes where speed is not null', con)
sql_result6

,index,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,424,N201AA,1959.0,Fixed wing single engine,CESSNA,150,1,2,90.0,Reciprocating
1,427,N202AA,1980.0,Fixed wing multi engine,CESSNA,421C,2,8,90.0,Reciprocating
2,821,N350AA,1980.0,Fixed wing multi engine,PIPER,PA-31-350,2,8,162.0,Reciprocating
3,893,N364AA,1973.0,Fixed wing multi engine,CESSNA,310Q,2,6,167.0,Reciprocating
4,1027,N378AA,1963.0,Fixed wing single engine,CESSNA,172E,1,4,105.0,Reciprocating
5,1037,N381AA,1956.0,Fixed wing multi engine,DOUGLAS,DC-7BF,4,102,232.0,Reciprocating
6,1190,N425AA,1968.0,Fixed wing single engine,PIPER,PA-28-180,1,4,107.0,Reciprocating
7,1430,N508AA,1975.0,Rotorcraft,BELL,206B,1,5,112.0,Turbo-shaft
8,1480,N519MQ,1983.0,Fixed wing single engine,CESSNA,A185F,1,6,127.0,Reciprocating
9,1515,N525AA,1980.0,Fixed wing multi engine,PIPER,PA-31-350,2,8,162.0,Reciprocating


In [112]:
frame6 = planes[~planes.speed.isnull()]
frame6

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
424,N201AA,1959.0,Fixed wing single engine,CESSNA,150,1,2,90.0,Reciprocating
427,N202AA,1980.0,Fixed wing multi engine,CESSNA,421C,2,8,90.0,Reciprocating
821,N350AA,1980.0,Fixed wing multi engine,PIPER,PA-31-350,2,8,162.0,Reciprocating
893,N364AA,1973.0,Fixed wing multi engine,CESSNA,310Q,2,6,167.0,Reciprocating
1027,N378AA,1963.0,Fixed wing single engine,CESSNA,172E,1,4,105.0,Reciprocating
1037,N381AA,1956.0,Fixed wing multi engine,DOUGLAS,DC-7BF,4,102,232.0,Reciprocating
1190,N425AA,1968.0,Fixed wing single engine,PIPER,PA-28-180,1,4,107.0,Reciprocating
1430,N508AA,1975.0,Rotorcraft,BELL,206B,1,5,112.0,Turbo-shaft
1480,N519MQ,1983.0,Fixed wing single engine,CESSNA,A185F,1,6,127.0,Reciprocating
1515,N525AA,1980.0,Fixed wing multi engine,PIPER,PA-31-350,2,8,162.0,Reciprocating


In [110]:
sql_result7 = pd.read_sql_query('select tailnum from planes where year >= 2010', con)
sql_result7

,tailnum
0,N127UW
1,N128UW
2,N150UW
3,N151UW
4,N152UW
5,N153UW
6,N154UW
7,N155UW
8,N156UW
9,N157UW


In [135]:
frame7 = pd.DataFrame(planes[['year','tailnum']][planes.year >= 2010]).reset_index(drop=True)
pd.DataFrame(frame7['tailnum']).equals(sql_result7)

True

In [139]:
sql_result8 = pd.read_sql_query('select tailnum from planes where seats between 100 and 200 limit 20', con)
sql_result8

,tailnum
0,N102UW
1,N103US
2,N104UW
3,N105UW
4,N107US
5,N108UW
6,N109UW
7,N110UW
8,N111US
9,N11206


In [153]:
frame8 = pd.DataFrame(planes[['tailnum','seats']][planes.seats >= 100][planes.seats <= 200]).reset_index(drop=True)['tailnum']
frame8.head(20)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


0     N102UW
1     N103US
2     N104UW
3     N105UW
4     N107US
5     N108UW
6     N109UW
7     N110UW
8     N111US
9     N11206
10    N112US
11    N113UW
12    N114UW
13    N117UW
14    N118US
15    N119US
16    N12109
17    N12114
18    N12116
19    N12125
Name: tailnum, dtype: object

In [155]:
sql_result9 = pd.read_sql_query('select * from planes where manufacturer in ("BOEING","AIRBUS","EMBRAER")',con)
sql_result9

,index,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,0,N10156,2004.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,None,Turbo-fan
1,4,N10575,2002.0,Fixed wing multi engine,EMBRAER,EMB-145LR,2,55,None,Turbo-fan
2,10,N11106,2002.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,None,Turbo-fan
3,11,N11107,2002.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,None,Turbo-fan
4,12,N11109,2002.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,None,Turbo-fan
5,13,N11113,2002.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,None,Turbo-fan
6,14,N11119,2002.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,None,Turbo-fan
7,15,N11121,2003.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,None,Turbo-fan
8,16,N11127,2003.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,None,Turbo-fan
9,17,N11137,2003.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,None,Turbo-fan


In [161]:
frame9 = pd.DataFrame(planes[planes.manufacturer.isin(["BOEING","AIRBUS","EMBRAER"])]).reset_index()
frame9

,index,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,0,N10156,2004.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
1,4,N10575,2002.0,Fixed wing multi engine,EMBRAER,EMB-145LR,2,55,NaN,Turbo-fan
2,10,N11106,2002.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
3,11,N11107,2002.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
4,12,N11109,2002.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
5,13,N11113,2002.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
6,14,N11119,2002.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
7,15,N11121,2003.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
8,16,N11127,2003.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan
9,17,N11137,2003.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan


In [163]:
sql_result10 = pd.read_sql_query('select * from planes where manufacturer in ("BOEING","AIRBUS","EMBRAER") and seats>300',con)
sql_result10

,index,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,49,N1200K,1998.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
1,50,N1201P,1998.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
2,69,N121DE,1987.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
3,117,N136DL,1991.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
4,122,N137DL,1991.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
5,183,N143DA,1998.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
6,241,N1602,1999.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
7,242,N1603,1999.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
8,243,N1604R,1999.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan
9,244,N1605,1999.0,Fixed wing multi engine,BOEING,767-332,2,330,None,Turbo-fan


In [166]:
frame10 = pd.DataFrame(planes[planes.manufacturer.isin(["BOEING","AIRBUS","EMBRAER"]) & (planes.seats > 300)]).reset_index()
frame10

,index,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,49,N1200K,1998.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
1,50,N1201P,1998.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
2,69,N121DE,1987.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
3,117,N136DL,1991.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
4,122,N137DL,1991.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
5,183,N143DA,1998.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
6,241,N1602,1999.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
7,242,N1603,1999.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
8,243,N1604R,1999.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan
9,244,N1605,1999.0,Fixed wing multi engine,BOEING,767-332,2,330,NaN,Turbo-fan


In [169]:
sql_result11 = pd.read_sql_query('select manufacturer, COUNT(*) from planes where seats>200 group by manufacturer',con)
sql_result11

,manufacturer,COUNT(*)
0,AIRBUS,66
1,AIRBUS INDUSTRIE,4
2,BOEING,225


In [202]:
frame11 = pd.DataFrame(planes[planes.seats > 200]).groupby('manufacturer').count()['tailnum'].reset_index()
frame11.columns = ['manufacturer','COUNT(*)']
frame11

,manufacturer,COUNT(*)
0,AIRBUS,66
1,AIRBUS INDUSTRIE,4
2,BOEING,225


In [203]:
sql_result13 = pd.read_sql_query('select manufacturer, COUNT(*) from planes where seats>200 group by manufacturer having count(*) > 10',con)
sql_result13

,manufacturer,COUNT(*)
0,AIRBUS,66
1,BOEING,225


In [204]:
frame13 = pd.DataFrame(planes[planes.seats > 200]).manufacturer.value_counts().reset_index()
frame13[frame13.manufacturer > 10]

,index,manufacturer
0,BOEING,225
1,AIRBUS,66
